In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [2]:
import torch

print(torch.cuda.is_available())

False


/mnt/apps/users/ikarunak/conda/envs/lcat/lib/python3.10/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
text_doc = "resources/images/lightfootcat/images/extracted_text.txt"
divisions = ["Dicotyledones", "Monocotyledones", "Pteridophyta", "Hepaticae", "Algae"]#
prompt = "resources/prompts/lightfootcat_prompt.yaml"

text_doc2 = "resources/images/hanbury/extracted_text.txt"
divisions2 = ["Dicotyledones", "GYMNOSPERMEAE", "Monocotyledones"]
prompt1 = "resources/prompts/hanbury_prompt.yaml"


In [4]:
with open(text_doc2, "r") as f:
    extracted_text = f.read()

In [5]:
extracted_text

'The letters of the alphabet under each species are used to indicate a herbarium sheet, and the numerals in parenthesis refer to the specimens from different localities.\n\nDICOTYLEDONES.\nI. POLYPETALAE. I. THALAMIFLORA.\nRANUNCULACEAE.\n1. Aconitum Napellus, L.\na. A portion of the plant in flower.\nHohneck, Vosges, 13 Aug., 1855. Pic de Sancy, Auvergne, Sept., 1858.\nb. Plant in flower, with root attached.\n2. Aconitum Lycoctonum, L.\na. Plant in flower, with root attached.\nHohneck, Vosges, 13 Aug., 1855.\n3. Actaea spicata, L.\na. Leafy stem with flowers and detached fruit. Cultivated at Clapham Common, 1867.\nb. Two leafy stems with fruit, and detached fruit.\n(1) Hohneck, Vosges, 13 Aug., 1855. (2) Auvergne, Sept., 1855.\n4. Anemone Hepatica, L.\na. Plant in flower, two specimens.\n(1) Salzburg, 10 Sept., 1862. With root attached.\n(2) Mont. Ventoux, near Avignon, 7 May, 1866.\n\nThe following extract is attached to this specimen:—\n\nAmong the dried specimens I send a portion o

In [6]:
import re

# extracted_text = re.sub(r"([A-Z]+)\s*(EAE|FAE|EAF)", r"\1EAE", extracted_text)
# extracted_text = re.sub(r"(\w)-\n+(\w)", "\1-\2", extracted_text)

In [7]:
from lib.data_processing.text_processing import TextProcessor
text_processor = TextProcessor()
#text_structure = text_processor(extracted_text, divisions=divisions, max_chunk_size=3000)
#text_blocks = text_processor.make_text_blocks(text_structure)

In [8]:
cleaned_text = text_processor.preprocess_text(extracted_text, divisions2[0])

print(cleaned_text)

DICOTYLEDONES.
I. POLYPETALAE. I. THALAMIFLORA.
RANUNCULACEAE
1. Aconitum Napellus, L.
a. A portion of the plant in flower.
Hohneck, Vosges, 13 Aug., 1855. Pic de Sancy, Auvergne, Sept., 1858.
b. Plant in flower, with root attached.
2. Aconitum Lycoctonum, L.
a. Plant in flower, with root attached.
Hohneck, Vosges, 13 Aug., 1855.
3. Actaea spicata, L.
a. Leafy stem with flowers and detached fruit. Cultivated at Clapham Common, 1867.
b. Two leafy stems with fruit, and detached fruit.
(1) Hohneck, Vosges, 13 Aug., 1855. (2) Auvergne, Sept., 1855.
4. Anemone Hepatica, L.
a. Plant in flower, two specimens.
(1) Salzburg, 10 Sept., 1862. With root attached.
(2) Mont. Ventoux, near Avignon, 7 May, 1866.

The following extract is attached to this specimen:—

Among the dried specimens I send a portion of a vine, or large creeper, called by the Indians 'Crespia,' or blistering leaf, much used among them and among many of the native doctors. I have often recommended it myself to the poor as a che

In [10]:
from lib.model.hf_models.mistral_model import MISTRAL_7B_INSTRUCT

model = MISTRAL_7B_INSTRUCT()
model.load()

Loading model for [mistralai/Mistral-7B-Instruct-v0.3] to device [cuda]


Loading checkpoint shards: 100%|██████████| 3/3 [00:27<00:00,  9.09s/it]


Loading processor for [mistralai/Mistral-7B-Instruct-v0.3] to device [cuda:0]
2025-07-29 10:28:04,005 - lib.model.hf_models.hf_model - WARNING - Pad token is None. Setting pad token to eos token.


In [11]:
chunks = [cleaned_text[i: i+4000] for i in range(0, len(cleaned_text), 4000)]

In [13]:
print(chunks[0])

DICOTYLEDONES.
I. POLYPETALAE. I. THALAMIFLORA.
RANUNCULACEAE
1. Aconitum Napellus, L.
a. A portion of the plant in flower.
Hohneck, Vosges, 13 Aug., 1855. Pic de Sancy, Auvergne, Sept., 1858.
b. Plant in flower, with root attached.
2. Aconitum Lycoctonum, L.
a. Plant in flower, with root attached.
Hohneck, Vosges, 13 Aug., 1855.
3. Actaea spicata, L.
a. Leafy stem with flowers and detached fruit. Cultivated at Clapham Common, 1867.
b. Two leafy stems with fruit, and detached fruit.
(1) Hohneck, Vosges, 13 Aug., 1855. (2) Auvergne, Sept., 1855.
4. Anemone Hepatica, L.
a. Plant in flower, two specimens.
(1) Salzburg, 10 Sept., 1862. With root attached.
(2) Mont. Ventoux, near Avignon, 7 May, 1866.

The following extract is attached to this specimen:—

Among the dried specimens I send a portion of a vine, or large creeper, called by the Indians 'Crespia,' or blistering leaf, much used among them and among many of the native doctors. I have often recommended it myself to the poor as a che

In [ ]:
conversation = lambda x, y: [dict(role="system", content=x),
                dict(role="user", content=y)]

system_prompt = ("You are an expert in identifying plant families, tribes and series and speceis names under each family from text.\n"
                 "Treat multiple familiy, tribe and series information as seperate entities.\n"
                 "Given the identified family, tribe, series and species names under each family convert the information into a JSON.\n"
                "Ensure no generation of text and to only identify what is in the text.\n"
                "If the category is not present in the text, do not include it in the JSON.\n"
                "Return only Family, Tribe, Series and Species names in the JSON.\n"
                "Do not include any other information in the JSON.\n"
                 )
user_prompt = lambda x: (f"Identify the family, tribe and series of the following text: {x}.\n"
               )

#respose = model(conversation=conversation(system_prompt, user_prompt))

In [ ]:
responses = [model(conversation=conversation(system_prompt, user_prompt(chunk))) for chunk in chunks[:4]]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [43]:
print(chunks[2])

.. a été introduite dans l'ile par le capitaine W. F. W. Owen et maintenant cultivée à Bois Chéri et en d'autre endroits." D.H.

c. Leaf bases, one with free and the other with over-lapping basal lobes.

Botanical Gardens, Trinidad, 1873.

A letter from Mr. H. Prestoe, of the Trinidad Botanical Gardens, states as follows:

"I am now able to inform you that stem and leaves having developed on our big plant of Cocculus palmatus, I find both the leaf forms you have sent me, in sketch 7th June, occur on the same plant and on the same stem. I enclose central bits of leaves showing the two forms."

BERBERIDEAE.

25. Berberis aristata, D. C.

a. Twig with detached fruits and leaves.

Khajjiar, 500 feet, near Churba; Baden H. Powell, Lahore, from Dr. J. L. Stewart, Aug., 1870.

26. Berberis Lycium, Royle.

a. Flowering branch with leaves.

From good sized bushes in the garden of Mr. B. H. Powell, Lahore, April, 1871; from Dr. J. L. Stewart.

b. Flowering branches.

Dalhousie, about 5,500 feet.

In [42]:
print(responses[2][0])

{
  "Family": "BERBERIDACEAE",
  "Tribe": null,
  "Series": null,
  "Species": ["Berberis aristata", "Berberis Lycium", "Berberis vulgaris", "Berberis vulgaris var.", "Sarracenia purpurea", "Chelidonium majus", "Hypecoum procumbens", "Papaver Argemone", "Papaver dubium", "Papaver hybridum", "Papaver Rhaeas", "Papaver somniferum", "Capparis aphylla", "Capparis spinosa", "Cistus salviæfolius", "Anchietae salutaris", "Viola cinerea", "Viola pedata", "Canella alba", "Cinnamodendron corticosum"]
}


In [9]:
div_struct = text_processor.split_by_divisions(cleaned_text, divisions)

for key, value in div_struct.items():
    print(f"{key}...")  # Print first 100 characters of each division

DICOTYLEDONES...
MONOCOTYLEDONES...


In [10]:
sample = div_struct["DICOTYLEDONES"]

In [11]:
import re
from pygbif import species
finds = re.finditer(text_processor.family_regex, sample)

find_matches = []

for find in finds:
    print(find)

<re.Match object; span=(0, 17), match='\nI. POLYPETALAE. '>
<re.Match object; span=(32, 48), match='.\nRANUNCULACEAE\n'>
<re.Match object; span=(2984, 3003), match='.\n\nMENISPERMACAE.\n\n'>
<re.Match object; span=(8584, 8600), match='\n\nBERBERIDEAE.\n\n'>
<re.Match object; span=(9467, 9486), match='.\n\nSARRACENIACEAE\n\n'>
<re.Match object; span=(9643, 9660), match='.\n\nPAPAVERACEAE\n\n'>
<re.Match object; span=(10508, 10524), match='.\n\nCAPPARIDAE.\n\n'>
<re.Match object; span=(10714, 10728), match='.\n\nCISTINEAE\n\n'>
<re.Match object; span=(10806, 10822), match='.\n\nVIOLARIEAE.\n\n'>
<re.Match object; span=(11159, 11175), match='.\n\nCANELLACEAE\n\n'>
<re.Match object; span=(12470, 12483), match='.\n\nBIXINEAE\n\n'>
<re.Match object; span=(15548, 15566), match='\n\nCARYOPHYLLEAE.\n\n'>
<re.Match object; span=(15844, 15862), match='.\n\nTAMARISCINEAE\n\n'>
<re.Match object; span=(16856, 16871), match='\n\nGUTTIFERAE.\n\n'>
<re.Match object; span=(17466, 17485), match='\n\nDIPT

In [12]:
# !pip install https://github.com/nleguillarme/taxonerd/releases/download/v1.5.4/en_ner_eco_md-1.1.0.tar.gz

In [13]:
!pip install https://github.com/nleguillarme/taxonerd/releases/download/v1.5.4/en_ner_eco_biobert-1.1.0.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.9/520.9 MB 7.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [14]:
import spacy

nlp = spacy.load("en_ner_eco_biobert")

/mnt/apps/users/ikarunak/conda/envs/lcat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/apps/users/ikarunak/conda/envs/lcat/lib/python3.10/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [15]:
doc = nlp(cleaned_text)

/mnt/apps/users/ikarunak/conda/envs/lcat/lib/python3.10/site-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


In [59]:
family_regex = (r"""(?P<family>[A-Z]+(ACEAE|FLORAE|EAE))""")

In [60]:
family_matches = [i for i in doc.ents if re.match(family_regex, i.text)]

In [61]:
len(family_matches)

104

In [43]:
list(doc.ents)

[POLYPETALAE,
 RANUNCULACEAE,
 Aconitum Napellus, L.,
 Aconitum Lycoctonum, L.,
 Actaea spicata, L.,
 Anemone Hepatica, L.,
 Clematis Flammula, L.,
 Clematis graveolens, Hook,
 Zarmiri,
 Coptis Teeta, Wall,
 Mishmi,
 Delphinium Consolida, L.,
 Helleborus lividus, Ait.,
 Helleborus niger, L.,
 Helleborus viridis, L.,
 Nigella arvensis, L.,
 Nigella Damascena, L.,
 Nigella sativa, L.,
 Xanthorrhiza apiifolia, L'Her.,
 Ditto,
 MENISPERMACAE,
 Abuta rufescens, Aublet.,
 Menispermacaeæ,
 Abutua,
 Abuta,
 Abuta rufescens, Aubl,,
 Chondodendron tomentosum, R. & P.,
 Menispermaceæ,
 Cissampelos,
 Pareira brava,
 Botryopsis platyphylla, Miers,
 Menispermaceæ,
 Abutua pequena,
 Pareira brava pequena,
 Botryopsis platyphylla, Miers,
 Abuta miuda,
 Butinha vel,
 Pareira brava miuda,
 Botryopsis platyphylla, Miers,
 Chondodendron tomentosum, R. & P.,
 Botryopsis platyphylla, Miers,
 Chondodendron tomentosum, R. & P.,
 Butinha vel,
 Pareira Brava miuda,
 Ditto,
 Abuta miuda,
 Chondrodendron,
 Butua 

In [62]:
family_names = []
for i in doc.ents:
    match_ = re.match(family_regex, i.text)

    if match_:
        print("="*20)
        print(match_.group("family"))
        print(match_.span())
        print(i.text)
        print("="*20)
        family_names.append(match_.group("family"))

RANUNCULACEAE
(0, 13)
RANUNCULACEAE
BERBERIDEAE
(0, 11)
BERBERIDEAE
SARRACENIACEAE
(0, 14)
SARRACENIACEAE
PAPAVERACEAE
(0, 12)
PAPAVERACEAE
VIOLARIEAE
(0, 10)
VIOLARIEAE
CANELLACEAE
(0, 11)
CANELLACEAE


BIXINEAE
(0, 8)
BIXINEAE
CARYOPHYLLEAE
(0, 13)
CARYOPHYLLEAE
TAMARISCINEAE
(0, 13)
TAMARISCINEAE
DIPTEROCARPEAE
(0, 14)
DIPTEROCARPEAE
MALVACEAE
(0, 9)
MALVACEAE
STERCULIACEAE
(0, 13)
STERCULIACEAE
TILIACEAE
(0, 9)
TILIACEAE
MALPIGHIACEAE
(0, 13)
MALPIGHIACEAE
ZYGOPHYLLACEAE
(0, 14)
ZYGOPHYLLACEAE
GERANIACEAE
(0, 11)
GERANIACEAE

TRIBE
PELARGONIEAE
(0, 12)
PELARGONIEAE
RUTACEAE
(0, 8)
RUTACEAE

TRIBE
RUTEAE
(0, 6)
RUTEAE
DIOSMEAE
(0, 8)
DIOSMEAE
SIMARUBACEAE
(0, 12)
SIMARUBACEAE
BURSERACEAE
(0, 11)
BURSERACEAE
AMPELIDEAE
(0, 10)
AMPELIDEAE
SAPINDACEAE
(0, 11)
SAPINDACEAE
ANACARDIACEAE
(0, 13)
ANACARDIACEAE

124
CORIARIACEAE
(0, 12)
CORIARIACEAE
MORINGACEAE
(0, 11)
MORINGACEAE
PAPILIONACEAE
(0, 13)
PAPILIONACEAE
ROSACEAE
(0, 8)
ROSACEAE
TRIBE
CHRYSOBALANEAE
(0, 14)
CHRYSOBALANEAE
POTENT

In [141]:
def get_family_spans(text, family_names):

    def get_batches(family_names, batch_size=5000):
        
        batches = []
        current_batch = ""
        current_length = 0
        for i in range(0, len(family_names)):
            family = family_names[i]
            if current_length + len(family) + 1 < batch_size:  # +1 for |
                current_length += len(family) + 1  # +1 for the separator
                current_batch += family + "|" if i + 1 < len(family_names) else family
            else:
                batches.append(current_batch)
                current_batch = family + "|" if i + 1 < len(family_names) else family
                current_length = len(current_batch)
        
        batches.append(current_batch)  # Add the last batch if it exists
        return batches            
        
    batches = get_batches(family_names)

    compiled_patterns = [re.compile(rf"^\b(?P<family>{batch})\b", flags=re.MULTILINE) for batch in batches]


    spans = []

    for pattern in compiled_patterns:
        for match in pattern.finditer(text):
            spans.append((match.group("family"), match.span()))

    return spans
family_names

['RANUNCULACEAE',
 'BERBERIDEAE',
 'SARRACENIACEAE',
 'PAPAVERACEAE',
 'VIOLARIEAE',
 'CANELLACEAE',
 'BIXINEAE',
 'CARYOPHYLLEAE',
 'TAMARISCINEAE',
 'DIPTEROCARPEAE',
 'MALVACEAE',
 'STERCULIACEAE',
 'TILIACEAE',
 'MALPIGHIACEAE',
 'ZYGOPHYLLACEAE',
 'GERANIACEAE',
 'PELARGONIEAE',
 'RUTACEAE',
 'RUTEAE',
 'DIOSMEAE',
 'SIMARUBACEAE',
 'BURSERACEAE',
 'AMPELIDEAE',
 'SAPINDACEAE',
 'ANACARDIACEAE',
 'CORIARIACEAE',
 'MORINGACEAE',
 'PAPILIONACEAE',
 'ROSACEAE',
 'CHRYSOBALANEAE',
 'POTENTILLEAE',
 'POTERIEAE',
 'HAMAMELIDEAE',
 'COMBRETACEAE',
 'MYRTACEAE',
 'PASSIFLORAEAE',
 'PAPAYACEAE',
 'FICOIDEAE',
 'HETEROSCIADEAE',
 'RUBIACEAE',
 'INULOIDEAE',
 'HELIANTHOIDEAE',
 'ANTHEMIDEAE',
 'SENECONIDEAE',
 'CYNAROIDEAE',
 'MUTISIACEAE',
 'CICHORIACEAE',
 'VACCINIACEAE',
 'ERICACEAE',
 'MYRSINEAE',
 'SAPOTACEAE',
 'EBENACEAE',
 'OLEACEAE',
 'SALVADORACEAE',
 'APOCYNACEAE',
 'ASCLEPIADACEAE',
 'LOGANIACEAE',
 'GENTIANACEAE',
 'BORAGINACEAE',
 'SOLANACEAE',
 'SCROPHULARINEAE',
 'OROBANCHACE

In [142]:
find_matches = get_family_spans(cleaned_text, family_names)

In [143]:
def get_tribes_series_matches(text: str):
    tribe_series_regex = r"((TRIBE|SERIES)\s+[IVXLCDM\.]+\s+[A-Z]+)"
    matches_ = re.finditer(tribe_series_regex, text)
    matches = [i for i in matches_]
    return matches


def split_by_families(text: str, find_matches: list[tuple[str, tuple[int, int]]]):

        text_chunks = []
        
        for idx, i in enumerate(find_matches):
            match = i[0]
            spans = i[1]
            start = spans[1]
            end = find_matches[idx+1][1][0] if idx+1 < len(find_matches) else None
            text_chunk = text[start:end] if end else text[start:]

            # ts_matches = get_tribes_series_matches(text_chunk)
            # if ts_matches:
            #     for ts_match in ts_matches:
            #         ts_text = ts_match.group(0)
            #         ts_start, ts_end = ts_match.span()
            #         text_chunk = text_chunk[:ts_start] + f"{ts_text} " + text_chunk[ts_end:]
            text_chunks.append(dict(family=match, text=text_chunk))
        
        return text_chunks

In [144]:
split_by_families(cleaned_text, find_matches)

[{'family': 'RANUNCULACEAE',
  'text': '\n1. Aconitum Napellus, L.\na. A portion of the plant in flower.\nHohneck, Vosges, 13 Aug., 1855. Pic de Sancy, Auvergne, Sept., 1858.\nb. Plant in flower, with root attached.\n2. Aconitum Lycoctonum, L.\na. Plant in flower, with root attached.\nHohneck, Vosges, 13 Aug., 1855.\n3. Actaea spicata, L.\na. Leafy stem with flowers and detached fruit. Cultivated at Clapham Common, 1867.\nb. Two leafy stems with fruit, and detached fruit.\n(1) Hohneck, Vosges, 13 Aug., 1855. (2) Auvergne, Sept., 1855.\n4. Anemone Hepatica, L.\na. Plant in flower, two specimens.\n(1) Salzburg, 10 Sept., 1862. With root attached.\n(2) Mont. Ventoux, near Avignon, 7 May, 1866.\n\nThe following extract is attached to this specimen:—\n\nAmong the dried specimens I send a portion of a vine, or large creeper, called by the Indians \'Crespia,\' or blistering leaf, much used among them and among many of the native doctors. I have often recommended it myself to the poor as a che

In [70]:
family_splits = split_by_families(cleaned_text, find_matches)

In [72]:
family_splits[0]["text"]

'RANUNCULACEAE\n1. Aconitum Napellus, L.\na. A portion of the plant in flower.\nHohneck, Vosges, 13 Aug., 1855. Pic de Sancy, Auvergne, Sept., 1858.\nb. Plant in flower, with root attached.\n2. Aconitum Lycoctonum, L.\na. Plant in flower, with root attached.\nHohneck, Vosges, 13 Aug., 1855.\n3. Actaea spicata, L.\na. Leafy stem with flowers and detached fruit. Cultivated at Clapham Common, 1867.\nb. Two leafy stems with fruit, and detached fruit.\n(1) Hohneck, Vosges, 13 Aug., 1855. (2) Auvergne, Sept., 1855.\n4. Anemone Hepatica, L.\na. Plant in flower, two specimens.\n(1) Salzburg, 10 Sept., 1862. With root attached.\n(2) Mont. Ventoux, near Avignon, 7 May, 1866.\n\nThe following extract is attached to this specimen:—\n\nAmong the dried specimens I send a portion of a vine, or large creeper, called by the Indians \'Crespia,\' or blistering leaf, much used among them and among many of the native doctors. I have often recommended it myself to the poor as a cheap blister. In using it, a

In [129]:
for i in family_splits:
    
    matches_ = re.finditer(tribe_species_regex, i["text"])
    matches = [i for i in matches_]
    if matches:
        print(f"Found {len(list(matches))} matches in family: {i['family']}")
    for match_ in matches:
        if match_:
            print(match_.group(0), match_.span())
    
    if matches:
        print("="*20)

Found 1 matches in family: PELARGONIEAE
TRIBE VI. OXALIDAE (390, 408)
Found 1 matches in family: DIOSMEAE
TRIBE V. XANTHOXYLAE (130, 150)
Found 1 matches in family: CHRYSOBALANEAE
TRIBE II. PRUNEA (69, 85)
Found 2 matches in family: POTERIEAE
TRIBE VIII. ROSEA (315, 332)
TRIBE X. POMAE (2352, 2366)
Found 2 matches in family: HETEROSCIADEAE
SERIES II. HAPLOZYGA (420, 440)
SERIES I. INFERAE (9257, 9274)
Found 1 matches in family: CICHORIACEAE
SERIES II. HETEROMERAE (424, 446)
Found 1 matches in family: EBENACEAE
SERIES III. BICARPELLATAE (647, 672)
Found 3 matches in family: BORAGINACEAE
TRIBE I. CORDIACE (13, 30)
TRIBE III. HELIOTROPIACE (530, 554)
TRIBE IV. BORAGINACE (705, 725)
Found 1 matches in family: POLYGONACEAE
SERIES III. MULTIOVULATAE (954, 979)
Found 1 matches in family: ARISTOLOCHIACEAE
SERIES IV. MICREM (809, 826)
Found 1 matches in family: PIPERACEAE
SERIES V. DAPHNALES (1185, 1204)
Found 1 matches in family: THYMELACEAE
SERIES VII. UNISEXUALES (340, 363)
Found 2 matches i